### Install required package

In [84]:

!pip install pymongo
!pip install python-dotenv

### Importing the required libraries

In [85]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os

In [88]:
# Load environment variables from .env file
load_dotenv()

MONGO_URI =  os.getenv("MONGO_URI")

In [90]:
# Create a new client and connect to the server
client = MongoClient(MONGO_URI, server_api=ServerApi('1'))
db = client['ehr_database']
collection = db['patient_records']
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("You successfully connected to MongoDB!")
except Exception as e:
    print(e)

You successfully connected to MongoDB!


### Function to fetch data from the database

In [72]:
from typing import Dict, Any
import json

def get_mental_health_data(patient_id: str) -> Dict[str, Any]:
    """Mock function to get mental health data"""
    return {
        "patient_id": patient_id,
        "diagnoses": ["anxiety", "depression"],
        "medications": ["sertraline", "alprazolam"],
        "therapies": ["cognitive behavioral therapy", "mindfulness"]
    }

In [92]:
def get_mental_health_data(patient_id: str) -> Dict[str, Any]:
    """Get patient mental health data from MongoDB"""
    try:
        patient = collection.find_one({"PatientID": int(patient_id)})
        if patient:
            encounters = patient.get("Encounters", [])
            encounter_history = [
                {
                    "date": e.get("Date"),
                    "reason": e.get("Reason"),
                    "notes": e.get("Notes")
                }
                for e in encounters
            ]
            
            return {
                "patient_id": str(patient["PatientID"]),
                "age": patient.get("Age"),
                "gender": patient.get("Gender"),
                "diagnosis": patient.get("Diagnosis"),
                "medications": patient.get("Medications", []),
                "encounter_history": encounter_history
            }
        return {"error": "Patient not found"}
    except Exception as e:
        return {"error": str(e)}

### Fetch data manually by function

In [95]:
get_mental_health_data("1002")

{'patient_id': '1002',
 'age': 31,
 'gender': 'Male',
 'diagnosis': 'OCD',
 'medications': ['Clomipramine 25mg', 'Sertraline 50mg'],
 'encounter_history': [{'date': '2024-03-31',
   'reason': 'Initial Consultation',
   'notes': 'Patient reports ritualistic actions and intrusive thoughts. Recommended medication adjustment.'},
  {'date': '2024-07-06',
   'reason': 'Medication Review',
   'notes': 'Patient reports intrusive thoughts and anxiety. Recommended medication adjustment.'}]}

### Define function call for tool calls

In [73]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "get_mental_health_ehr_data",
          "parameters": {
              "type": "object",
              "properties": {
                  "patient_id": {"type": "string"}
              },
          },
      },
  }
]

### Load environment variables from .env file

In [116]:
from openai import OpenAI
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

### Prepare prompt with function calling

In [122]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "What are the recent mental health notes for patient #1012?"}],
  tools=tools,
)

response = completion.choices[0].message.tool_calls
print(response)

[ChatCompletionMessageToolCall(id='call_CAAOxNEwb6fPFVOzwLzShgzc', function=Function(arguments='{"patient_id":"1012"}', name='get_mental_health_ehr_data'), type='function')]


### Test the function call accuracy

In [126]:
import random

# Prompt templates with {patient_id} placeholder
prompt_templates = [
    "What are the recent mental health notes for patient #{patient_id}?",
    "Show me the treatment history and medications for patient {patient_id}",
    "Get the latest diagnosis and encounters for patient ID {patient_id}",
    "What is the mental health progress for patient #{patient_id}?",
    "List all medications and treatment notes for patient {patient_id}",
    "Provide mental health assessment history for patient #{patient_id}"
]


def test_function_call_accuracy(num_tests=10):
    successful_calls = 0
    total_calls = num_tests
    
    print(f"Running {num_tests} function call tests...")
    print("-" * 50)
    
    for i in range(num_tests):
        # Generate random patient ID between 1000-1099
        patient_id = str(random.randint(1000, 1099))
        # Select random prompt template
        prompt = random.choice(prompt_templates).format(patient_id=patient_id)
        
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            tools=tools,
        )
        
        response = completion.choices[0].message.tool_calls
        if response:
            function_name = response[0].function.name
            if function_name == "get_mental_health_ehr_data":
                successful_calls += 1
                print(f"Test {i+1}: ✓ Correct function call")
            else:
                print(f"Test {i+1}: ✗ Wrong function called: {function_name}")
        else:
            print(f"Test {i+1}: ✗ No function called")
    accuracy = (successful_calls / total_calls) * 100
    print("-" * 50)
    print(f"Total tests: {total_calls}")
    print(f"Successful calls: {successful_calls}")
    print(f"Accuracy: {accuracy:.2f}%")

# Usage:
test_function_call_accuracy(100)

Running 100 function call tests...
--------------------------------------------------
Test 1: ✓ Correct function call
Test 2: ✓ Correct function call
Test 3: ✓ Correct function call
Test 4: ✓ Correct function call
Test 5: ✓ Correct function call
Test 6: ✓ Correct function call
Test 7: ✓ Correct function call
Test 8: ✓ Correct function call
Test 9: ✓ Correct function call
Test 10: ✓ Correct function call
Test 11: ✓ Correct function call
Test 12: ✓ Correct function call
Test 13: ✓ Correct function call
Test 14: ✓ Correct function call
Test 15: ✓ Correct function call
Test 16: ✓ Correct function call
Test 17: ✓ Correct function call
Test 18: ✓ Correct function call
Test 19: ✓ Correct function call
Test 20: ✓ Correct function call
Test 21: ✓ Correct function call
Test 22: ✓ Correct function call
Test 23: ✓ Correct function call
Test 24: ✓ Correct function call
Test 25: ✓ Correct function call
Test 26: ✓ Correct function call
Test 27: ✓ Correct function call
Test 28: ✓ Correct function call

### Test DB call

In [123]:
function_name = response[0].function.name
function_name

if function_name == "get_mental_health_ehr_data":
    parameters = response[0].function.arguments
    parameters_dict = json.loads(parameters)
    patient_id = parameters_dict["patient_id"]
    result = get_mental_health_data(patient_id)
    print(json.dumps(result, indent=2))


{
  "patient_id": "1012",
  "age": 34,
  "gender": "Male",
  "diagnosis": "PTSD",
  "medications": [
    "Sertraline 100mg",
    "Paroxetine 20mg",
    "Prazosin 2mg"
  ],
  "encounter_history": [
    {
      "date": "2024-03-19",
      "reason": "Initial Consultation",
      "notes": "Patient reports hypervigilance and flashbacks. Recommended group therapy."
    },
    {
      "date": "2024-04-28",
      "reason": "Follow-up",
      "notes": "Patient reports flashbacks and nightmares. Recommended stress management techniques."
    },
    {
      "date": "2024-05-25",
      "reason": "Medication Review",
      "notes": "Patient reports avoidance and nightmares. Recommended CBT."
    },
    {
      "date": "2024-07-15",
      "reason": "Follow-up",
      "notes": "Patient reports flashbacks and nightmares. Recommended mindfulness training."
    }
  ]
}
